In [1]:
import tensorflow as tf
# import tensorflow_hub as hub
import os
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
#from keras import optimizers
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
# import matplotlib.pyplot as plt
from glob import glob

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

UsageError: Line magic function `%tensorflow_version` not found.


In [3]:
#Set the image size as per the VGG16 archetecture
IMAGE_SIZE = [256, 256]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls '/content/drive'

MyDrive


In [4]:
#Give dataset path
train_path = 'X:\\Projects\\segmented\\train'
# validation_path = 'X:\\Projects\\tomato\\val'
test_path = 'X:\\Projects\\segmented\\test'

In [5]:
import time
import os
from os.path import exists

def count(dir, counter=0):
    "returns number of files in dir and subdirs"
    for pack in os.walk(dir):
        for f in pack[2]:
            counter += 1
    return dir + " : " + str(counter) + "files"

In [6]:
print('total images for training :', count(train_path))
# print('total images for validation :', count(validation_path))
print('total images for validation :', count(test_path))

total images for training : X:\Projects\segmented\train : 7460files
total images for validation : X:\Projects\segmented\test : 1865files


In [7]:
folders = glob('X:\\Projects\\segmented\\train\\*')
print(len(folders))


10


In [16]:
BATCH_SIZE = 32

train_generator = ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.1) # VGG16 preprocessing

# validation_generator = ImageDataGenerator(preprocessing_function=preprocess_input) 

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessin

In [17]:
class_subset = sorted(os.listdir('X:\\Projects\\segmented\\train'))
print(class_subset)
traingen = train_generator.flow_from_directory(train_path,
                                               target_size=(256, 256),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=5)

validgen = train_generator.flow_from_directory(train_path,
                                               target_size=(256, 256),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=4)

testgen = test_generator.flow_from_directory(test_path,
                                             target_size=(256, 256),
                                             class_mode=None,
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=4)

['bacterialspot', 'earlyblight', 'healthy', 'lateblight', 'leafmold', 'mosaicvirus', 'septorialeafspot', 'spidermites', 'targetspot', 'yellowleafcurlvirus']
Found 6715 images belonging to 10 classes.
Found 745 images belonging to 10 classes.
Found 1865 images belonging to 10 classes.


# **VGG16**

In [18]:
from keras.applications.vgg16 import VGG16
model1 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [19]:
for layer in model1.layers:
  layer.trainable = False

In [20]:
model1.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [24]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
batnorm = layers.BatchNormalization()
dropout = layers.Dropout(0.20)
prediction_layer = layers.Dense(10, activation='sigmoid')


vgg16 = models.Sequential([
    model1,
    flatten_layer,
    batnorm,
    dropout,
    prediction_layer
])

In [25]:
vgg16.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)             0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32768)             131072    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32768)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                327690    
Total params: 15,173,450
Trainable params: 393,226
Non-trainable params: 14,780,224
_________________________________________________________________


In [26]:
optimizer=Adam(learning_rate=0.001)
vgg16.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


from keras.callbacks import ModelCheckpoint, EarlyStopping
optim_1 = Adam(learning_rate=0.001)
#n_classes=3

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 50

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='X:\\Projects\\segmented\\vgg16.h5',
                                  save_best_only=True,
                                  monitor='val_accuracy',
                                  mode='max',
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')


vgg_history = vgg16.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[tl_checkpoint_1], 
                            verbose=1)

Epoch 1/50
209/209 [==============================] - 79s 376ms/step - loss: 0.8377 - accuracy: 0.7510 - val_loss: 0.6753 - val_accuracy: 0.8370

Epoch 00001: val_accuracy improved from -inf to 0.83696, saving model to X:\Projects\segmented\vgg16.h5
Epoch 2/50
209/209 [==============================] - 80s 381ms/step - loss: 0.1187 - accuracy: 0.9602 - val_loss: 0.6599 - val_accuracy: 0.8655

Epoch 00002: val_accuracy improved from 0.83696 to 0.86549, saving model to X:\Projects\segmented\vgg16.h5
Epoch 3/50
209/209 [==============================] - 80s 383ms/step - loss: 0.0518 - accuracy: 0.9825 - val_loss: 0.8867 - val_accuracy: 0.8546

Epoch 00003: val_accuracy did not improve from 0.86549
Epoch 4/50
209/209 [==============================] - 80s 384ms/step - loss: 0.0386 - accuracy: 0.9877 - val_loss: 0.7835 - val_accuracy: 0.8668

Epoch 00004: val_accuracy improved from 0.86549 to 0.86685, saving model to X:\Projects\segmented\vgg16.h5
Epoch 5/50
209/209 [=======================

209/209 [==============================] - 80s 382ms/step - loss: 0.0393 - accuracy: 0.9934 - val_loss: 2.5948 - val_accuracy: 0.8383

Epoch 00041: val_accuracy did not improve from 0.86685
Epoch 42/50
209/209 [==============================] - 80s 382ms/step - loss: 0.0272 - accuracy: 0.9945 - val_loss: 2.4717 - val_accuracy: 0.8519

Epoch 00042: val_accuracy did not improve from 0.86685
Epoch 43/50
209/209 [==============================] - 80s 381ms/step - loss: 0.0233 - accuracy: 0.9945 - val_loss: 2.4661 - val_accuracy: 0.8438

Epoch 00043: val_accuracy did not improve from 0.86685
Epoch 44/50
209/209 [==============================] - 80s 382ms/step - loss: 0.0502 - accuracy: 0.9915 - val_loss: 2.7443 - val_accuracy: 0.8438

Epoch 00044: val_accuracy did not improve from 0.86685
Epoch 45/50
209/209 [==============================] - 78s 375ms/step - loss: 0.0388 - accuracy: 0.9939 - val_loss: 2.3924 - val_accuracy: 0.8424

Epoch 00045: val_accuracy did not improve from 0.86685
Ep

In [27]:
import numpy as np
# Generate predictions
vgg16.load_weights('X:\\Projects\\segmented\\vgg16.h5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

vgg_preds = vgg16.predict(testgen)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)

In [28]:
from sklearn.metrics import accuracy_score

vgg_acc = accuracy_score(true_classes, vgg_pred_classes)
print("VGG16 Model Accuracy without Fine-Tuning: {:.2f}%".format(vgg_acc * 100))

VGG16 Model Accuracy without Fine-Tuning: 83.70%


In [29]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(true_classes,vgg_pred_classes))

              precision    recall  f1-score   support

           0       0.83      0.88      0.85       200
           1       0.67      0.69      0.68       200
           2       0.86      0.87      0.87       200
           3       0.87      0.81      0.84       200
           4       0.86      0.86      0.86       190
           5       0.88      0.91      0.89        75
           6       0.83      0.75      0.79       200
           7       0.90      0.93      0.91       200
           8       0.74      0.79      0.77       200
           9       0.97      0.93      0.95       200

    accuracy                           0.84      1865
   macro avg       0.84      0.84      0.84      1865
weighted avg       0.84      0.84      0.84      1865



In [30]:
print(confusion_matrix(true_classes,vgg_pred_classes))

[[176   5   0   1   4   2   7   0   4   1]
 [ 11 137   5  15   6   2  11   3   9   1]
 [  1   3 174   0   1   1   0   3  17   0]
 [  4  16   3 163   8   0   3   0   3   0]
 [  0   7   5   2 164   1   0   3   5   3]
 [  0   1   0   1   2  68   1   1   0   1]
 [ 10  24   2   4   2   2 150   0   6   0]
 [  0   3   0   0   1   0   1 185  10   0]
 [ 10   5  13   1   2   0   6   5 158   0]
 [  1   3   0   1   0   1   1   6   1 186]]


# **ResNet152**

In [31]:
from keras.models import Model
import keras
model6 = tf.keras.applications.ResNet152(include_top=False, weights='imagenet', input_shape=(256,256,3))
for layer in model6.layers:
    layer.trainable = False

In [32]:
model6.summary()

Model: "resnet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

In [33]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
batnorm = layers.BatchNormalization()
dropout = layers.Dropout(0.20)
prediction_layer = layers.Dense(10, activation='sigmoid')

resnet152 = models.Sequential([
    model6,
    flatten_layer,
    batnorm,
    dropout,
    prediction_layer
])

In [34]:
resnet152.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152 (Functional)       (None, 8, 8, 2048)        58370944  
_________________________________________________________________
flatten_3 (Flatten)          (None, 131072)            0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 131072)            524288    
_________________________________________________________________
dropout_3 (Dropout)          (None, 131072)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1310730   
Total params: 60,205,962
Trainable params: 1,572,874
Non-trainable params: 58,633,088
_________________________________________________________________


In [35]:
optimizer=Adam(learning_rate=0.001)
resnet152.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


from keras.callbacks import ModelCheckpoint, EarlyStopping
optim_1 = Adam(learning_rate=0.001)
#n_classes=3

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 50

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='X:\\Projects\\segmented\\resnet152.h5',
                                  save_best_only=True,
                                  monitor='val_accuracy',
                                  mode='max',
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')


vgg_history = resnet152.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[tl_checkpoint_1], 
                            verbose=1)

Epoch 1/50
209/209 [==============================] - 130s 570ms/step - loss: 1.8205 - accuracy: 0.7815 - val_loss: 1.2817 - val_accuracy: 0.8668

Epoch 00001: val_accuracy improved from -inf to 0.86685, saving model to X:\Projects\segmented\resnet152.h5


C:\Users\bhala\anaconda3\envs\NNDL\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/50
209/209 [==============================] - 121s 577ms/step - loss: 0.5258 - accuracy: 0.9439 - val_loss: 1.2145 - val_accuracy: 0.9008

Epoch 00002: val_accuracy improved from 0.86685 to 0.90082, saving model to X:\Projects\segmented\resnet152.h5
Epoch 3/50
209/209 [==============================] - 120s 574ms/step - loss: 0.2215 - accuracy: 0.9695 - val_loss: 2.2042 - val_accuracy: 0.8954

Epoch 00003: val_accuracy did not improve from 0.90082
Epoch 4/50
209/209 [==============================] - 122s 585ms/step - loss: 0.2315 - accuracy: 0.9722 - val_loss: 2.5798 - val_accuracy: 0.8886

Epoch 00004: val_accuracy did not improve from 0.90082
Epoch 5/50
209/209 [==============================] - 123s 589ms/step - loss: 0.1687 - accuracy: 0.9789 - val_loss: 1.9965 - val_accuracy: 0.9062

Epoch 00005: val_accuracy improved from 0.90082 to 0.90625, saving model to X:\Projects\segmented\resnet152.h5
Epoch 6/50
209/209 [==============================] - 122s 584ms/step - loss: 0.

209/209 [==============================] - 120s 575ms/step - loss: 0.1244 - accuracy: 0.9933 - val_loss: 7.0936 - val_accuracy: 0.9076

Epoch 00041: val_accuracy did not improve from 0.91304
Epoch 42/50
209/209 [==============================] - 120s 576ms/step - loss: 0.1777 - accuracy: 0.9928 - val_loss: 5.9002 - val_accuracy: 0.9076

Epoch 00042: val_accuracy did not improve from 0.91304
Epoch 43/50
209/209 [==============================] - 121s 579ms/step - loss: 0.1469 - accuracy: 0.9940 - val_loss: 5.4605 - val_accuracy: 0.9035

Epoch 00043: val_accuracy did not improve from 0.91304
Epoch 44/50
209/209 [==============================] - 123s 588ms/step - loss: 0.1029 - accuracy: 0.9948 - val_loss: 6.6051 - val_accuracy: 0.9035

Epoch 00044: val_accuracy did not improve from 0.91304
Epoch 45/50
209/209 [==============================] - 124s 593ms/step - loss: 0.1275 - accuracy: 0.9942 - val_loss: 6.4566 - val_accuracy: 0.9117

Epoch 00045: val_accuracy did not improve from 0.913

In [36]:
import numpy as np
# Generate predictions
resnet152= tf.keras.models.load_model('X:\\Projects\\segmented\\resnet152.h5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

resnet152_preds = resnet152.predict(testgen)
resnet152_pred_classes = np.argmax(resnet152_preds, axis=1)

In [37]:
from sklearn.metrics import accuracy_score

resnet152_acc = accuracy_score(true_classes, resnet152_pred_classes)
print("ResNet152 Model Accuracy without Fine-Tuning: {:.2f}%".format(resnet152_acc * 100))

ResNet152 Model Accuracy without Fine-Tuning: 36.35%


In [38]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(true_classes,resnet152_pred_classes))

              precision    recall  f1-score   support

           0       0.29      0.99      0.45       200
           1       0.30      0.52      0.38       200
           2       0.47      0.66      0.55       200
           3       0.48      0.35      0.41       200
           4       0.39      0.35      0.37       190
           5       0.34      0.28      0.31        75
           6       0.14      0.05      0.07       200
           7       0.69      0.29      0.41       200
           8       0.41      0.04      0.06       200
           9       1.00      0.04      0.09       200

    accuracy                           0.36      1865
   macro avg       0.45      0.36      0.31      1865
weighted avg       0.46      0.36      0.31      1865



In [39]:
confusion_matrix(true_classes,resnet152_pred_classes)

array([[199,   0,   0,   1,   0,   0,   0,   0,   0,   0],
       [ 94, 104,   2,   0,   0,   0,   0,   0,   0,   0],
       [ 47,  21, 132,   0,   0,   0,   0,   0,   0,   0],
       [ 37,  72,  20,  71,   0,   0,   0,   0,   0,   0],
       [ 44,  24,  17,  38,  66,   0,   0,   1,   0,   0],
       [  5,  25,   2,   3,  19,  21,   0,   0,   0,   0],
       [113,  43,   8,   9,   9,   8,  10,   0,   0,   0],
       [ 19,  14,  42,   4,  29,   9,  24,  59,   0,   0],
       [ 89,  30,  36,   8,   3,   2,  14,  11,   7,   0],
       [ 34,  13,  20,  13,  44,  21,  21,  15,  10,   9]], dtype=int64)